In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from scipy.fftpack import dct, idct
import pywt
from scipy.stats import pearsonr
import warnings
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML # Added HTML
import json
import datetime
import hashlib
import os
from io import BytesIO # Needed for handling uploads
import base64

warnings.filterwarnings('ignore')

# --- Styling (Keep as before) ---
display(HTML("""
<style>
.output_png { display: table-cell; text-align: center; vertical-align: middle; }
.widget-label { font-weight: bold !important; color: #1f77b4 !important; min-width: 120px !important; }
.success { color: green; font-weight: bold; }
.error { color: red; font-weight: bold; }
.warning { color: orange; font-weight: bold; }
h3 { color: #1f77b4; border-bottom: 1px solid #ccc; padding-bottom: 5px; }
.widget-box { border: 1px solid #ccc !important; padding: 15px !important; margin-bottom: 10px !important; border-radius: 5px !important; }
.widget-button { margin-top: 10px !important; }
</style>
"""))

print("🔍 Enhanced DWT+DCT+SVD Watermarking System with Copyright Management (v3 + GUI)")

🔍 Enhanced DWT+DCT+SVD Watermarking System with Copyright Management (v3 + GUI)


In [3]:
class FixedWatermarkingSystem:
    def __init__(self, wavelet='haar', block_size=8, base_alpha=0.1):
        """
        Fixed watermarking system comparing singular values (additive embedding).
        """
        self.wavelet = wavelet
        self.block_size = block_size
        self.base_alpha = base_alpha
        self.database = {}
        self.verification_log = []

    def generate_copyright_info(self, owner_name, contact_info, rights="All rights reserved"):
        """Generate copyright information"""
        timestamp = datetime.datetime.now().isoformat()
        copyright_id = hashlib.md5(f"{owner_name}{timestamp}".encode()).hexdigest()[:16]

        return {
            'copyright_id': copyright_id,
            'owner_name': owner_name,
            'contact_info': contact_info,
            'rights': rights,
            'timestamp': timestamp,
            'registration_date': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

    def text_to_watermark(self, text, size=(64, 64)):
        """Convert text to binary watermark pattern"""
        watermark = np.zeros(size, dtype=np.float64)
        text_binary = ''.join(format(ord(c), '08b') for c in text)

        height, width = size
        max_bits = height * width

        if len(text_binary) > max_bits:
            print(f"   ⚠️ Warning: Copyright text too long, truncated to {max_bits} bits.")
            text_binary = text_binary[:max_bits]

        for i, bit in enumerate(text_binary):
             if i < max_bits:
                row = i // width
                col = i % width
                watermark[row, col] = float(bit)

        return watermark

    def preprocess_image(self, image_path_or_bytesio, target_size=(512, 512)):
        """Preprocess image (path or BytesIO) for watermarking"""
        try:
            # Handle BytesIO from uploads or string path
            if isinstance(image_path_or_bytesio, BytesIO):
                image_path_or_bytesio.seek(0) # Go to start of stream
                img = Image.open(image_path_or_bytesio).convert('L')
            elif isinstance(image_path_or_bytesio, str):
                img = Image.open(image_path_or_bytesio).convert('L')
            else:
                 raise ValueError("Invalid image input type")

            img = np.array(img)
            if img.shape != target_size:
                 img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)

            if img.dtype != np.float64:
                 img = img.astype(np.float64)
            if img.max() > 1.0:
                 img = img / 255.0

            return np.clip(img, 0.0, 1.0)
        except Exception as e:
            print(f"   ❌ Error preprocessing image: {e}")
            raise

    def calculate_complexity(self, image):
        img_uint8 = (np.clip(image, 0, 1) * 255).astype(np.uint8)
        edges = cv2.Canny(img_uint8, 50, 150)
        return np.sum(edges > 0) / edges.size

    def adaptive_alpha(self, complexity):
        alpha = self.base_alpha * (1 + complexity * 1.5)
        return np.clip(alpha, 0.01, 0.2)

    def embed_watermark(self, host_image, watermark_pattern, alpha):
        print("   🔧 Embedding watermark...")
        coeffs_h = pywt.wavedec2(host_image, self.wavelet, level=1)
        LL_h, (HL_h, LH_h, HH_h) = coeffs_h
        wm_resized = cv2.resize(watermark_pattern, LL_h.shape[::-1], interpolation=cv2.INTER_NEAREST)
        coeffs_w = pywt.wavedec2(wm_resized, self.wavelet, level=1)
        LL_w, (HL_w, LH_w, HH_w) = coeffs_w
        LL_h_dct = self.block_dct(LL_h)
        try:
            U_h, S_h_original, Vt_h = np.linalg.svd(LL_h_dct, full_matrices=False)
        except np.linalg.LinAlgError:
            print("   ⚠️ SVD did not converge (host embed). Adding noise.")
            LL_h_dct += np.random.normal(0, 1e-10, LL_h_dct.shape)
            U_h, S_h_original, Vt_h = np.linalg.svd(LL_h_dct, full_matrices=False)
        try:
            U_w, S_w_original, Vt_w = np.linalg.svd(LL_w, full_matrices=False)
        except np.linalg.LinAlgError:
             print("   ⚠️ SVD did not converge (wm embed). Adding noise.")
             LL_w += np.random.normal(0, 1e-10, LL_w.shape)
             U_w, S_w_original, Vt_w = np.linalg.svd(LL_w, full_matrices=False)
        min_len = min(len(S_h_original), len(S_w_original))
        S_embedded = S_h_original.copy()
        S_embedded[:min_len] = S_h_original[:min_len] + alpha * S_w_original[:min_len] # Additive
        S_embedded_diag = np.zeros(LL_h_dct.shape, dtype=float)
        np.fill_diagonal(S_embedded_diag, S_embedded)
        U_h_use = U_h[:, :len(S_embedded)]
        Vt_h_use = Vt_h[:len(S_embedded), :]
        LL_h_dct_embedded = U_h_use @ S_embedded_diag[:len(S_embedded), :len(S_embedded)] @ Vt_h_use
        LL_h_embedded = self.inverse_block_dct(LL_h_dct_embedded)
        coeffs_embedded = [LL_h_embedded, (HL_h, LH_h, HH_h)]
        watermarked = pywt.waverec2(coeffs_embedded, self.wavelet)
        watermarked = np.clip(watermarked, 0, 1)
        extraction_data = {'S_h_original': S_h_original[:min_len],'S_w_original': S_w_original[:min_len],'alpha': alpha,'min_len': min_len,'wavelet': self.wavelet,'original_wm_pattern_shape': watermark_pattern.shape}
        return watermarked, extraction_data

    def extract_watermark(self, test_image, extraction_data):
        try:
            coeffs_t = pywt.wavedec2(test_image, extraction_data['wavelet'], level=1)
            LL_t, _ = coeffs_t
            LL_t_dct = self.block_dct(LL_t)
            try:
                U_t, S_t, Vt_t = np.linalg.svd(LL_t_dct, full_matrices=False)
            except np.linalg.LinAlgError:
                 print("   ⚠️ SVD did not converge during extraction.")
                 return None
            S_h_original = extraction_data['S_h_original']
            alpha = extraction_data['alpha']
            min_len = extraction_data['min_len']
            len_S_t = len(S_t)
            if len_S_t < min_len:
                 S_t_padded = np.pad(S_t, (0, min_len - len_S_t))
                 S_t_use = S_t_padded[:min_len]
            else:
                 S_t_use = S_t[:min_len]
            alpha_safe = alpha if alpha != 0 else 1e-10
            S_extracted = (S_t_use - S_h_original) / alpha_safe # Additive extraction
            S_extracted = np.nan_to_num(S_extracted)
            return S_extracted
        except Exception as e:
            print(f"   ❌ Extraction error: {str(e)}")
            import traceback; traceback.print_exc()
            return None

    def block_dct(self, subband):
        h, w = subband.shape
        dct_blocks = np.zeros_like(subband)
        pad_h = (self.block_size - h % self.block_size) % self.block_size
        pad_w = (self.block_size - w % self.block_size) % self.block_size
        subband_padded = np.pad(subband, ((0, pad_h), (0, pad_w)), mode='constant')
        hp, wp = subband_padded.shape
        for i in range(0, hp, self.block_size):
            for j in range(0, wp, self.block_size):
                block = subband_padded[i:i+self.block_size, j:j+self.block_size]
                if block.shape == (self.block_size, self.block_size):
                     dct_blocks[i:i+self.block_size, j:j+self.block_size] = dct(dct(block.T, norm='ortho').T, norm='ortho')
        return dct_blocks[:h, :w]

    def inverse_block_dct(self, dct_blocks):
        h, w = dct_blocks.shape
        idct_blocks = np.zeros_like(dct_blocks)
        pad_h = (self.block_size - h % self.block_size) % self.block_size
        pad_w = (self.block_size - w % self.block_size) % self.block_size
        dct_blocks_padded = np.pad(dct_blocks, ((0, pad_h), (0, pad_w)), mode='constant')
        hp, wp = dct_blocks_padded.shape
        for i in range(0, hp, self.block_size):
            for j in range(0, wp, self.block_size):
                block = dct_blocks_padded[i:i+self.block_size, j:j+self.block_size]
                if block.shape == (self.block_size, self.block_size):
                     idct_blocks[i:i+self.block_size, j:j+self.block_size] = idct(idct(block.T, norm='ortho').T, norm='ortho')
        return idct_blocks[:h, :w]

    def register_copyright(self, image_input, owner_name, contact_info, rights="All rights reserved"): # Accepts path or BytesIO
        print("📝 Starting copyright registration...")
        copyright_info = self.generate_copyright_info(owner_name, contact_info, rights)
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        copyright_text = f"COPYRIGHT:{copyright_info['copyright_id']}:{owner_name}:{timestamp}"
        watermark_pattern = self.text_to_watermark(copyright_text)
        try:
             host_image = self.preprocess_image(image_input) # Use image_input
        except Exception as e:
             print(f"   ❌ Failed to register: {e}")
             return None, None
        complexity = self.calculate_complexity(host_image)
        alpha = self.adaptive_alpha(complexity)
        print(f"📊 Image Analysis:\n   Complexity: {complexity:.4f}\n   Adaptive Alpha: {alpha:.4f}")
        watermarked_image, extraction_data = self.embed_watermark(host_image, watermark_pattern, alpha)
        self.database[copyright_info['copyright_id']] = {
            'copyright_info': copyright_info, 'extraction_data': extraction_data, 'original_host_shape': host_image.shape,
        }
        print("✅ Copyright registration completed!")
        print(f"📄 Copyright ID: {copyright_info['copyright_id']}")
        return watermarked_image, copyright_info

    def verify_authenticity(self, image_input, expected_owner=None): # Accepts path or BytesIO
        print("🔍 Starting authenticity verification...")
        try:
            test_image = self.preprocess_image(image_input) # Use image_input
        except Exception as e:
            print(f"   ❌ Verification failed: Could not process test image - {e}")
            report = self.generate_verification_report(None, 0, expected_owner, status_override="ERROR")
            self.log_verification(str(image_input), report) # Log input identifier
            return report, None, None

        best_match_info = None
        best_similarity = -1
        if not self.database:
             print("   ⚠️ Database is empty. Cannot verify.")
             report = self.generate_verification_report(None, 0, expected_owner, status_override="DB_EMPTY")
             self.log_verification(str(image_input), report)
             return report, None, None

        for copyright_id, data in self.database.items():
            try:
                if 'extraction_data' not in data: continue
                extracted_sv = self.extract_watermark(test_image, data['extraction_data'])
                if extracted_sv is not None:
                    original_sv = data['extraction_data']['S_w_original']
                    similarity = self.calculate_similarity(original_sv, extracted_sv)
                    print(f"   🔍 Testing against {copyright_id}: Similarity (Correlation) = {similarity:.4f}")
                    if similarity > best_similarity:
                        best_similarity = similarity
                        best_match_info = data['copyright_info']
                else: print(f"   ❌ Extraction failed for {copyright_id}")
            except KeyError as e: print(f"   ⚠️ Data format error for {copyright_id}: Missing key {e}. Skipping.")
            except Exception as e: print(f"   ⚠️ Error during verification against {copyright_id}: {str(e)}"); import traceback; traceback.print_exc()

        report = self.generate_verification_report(best_match_info, best_similarity, expected_owner)
        self.log_verification(str(image_input), report) # Log input identifier
        return report, best_match_info, None # No spatial WM returned

    def calculate_similarity(self, sv_original, sv_extracted):
        if sv_extracted is None: return 0.0
        len_orig, len_extr = len(sv_original), len(sv_extracted)
        if len_orig == 0 or len_extr == 0: return 0.0
        compare_len = min(len_orig, len_extr)
        if compare_len < len_orig or compare_len < len_extr: print(f"   ⚠️ Warning: SV lengths differ ({len_orig} vs {len_extr}). Comparing first {compare_len}.")
        vec1, vec2 = sv_original[:compare_len], sv_extracted[:compare_len]
        if np.std(vec1) == 0 or np.std(vec2) == 0: return 1.0 if np.allclose(vec1, vec2) else 0.0
        try: correlation, _ = pearsonr(vec1, vec2); return np.nan_to_num(correlation)
        except ValueError as e: print(f"   ⚠️ Error calculating correlation: {e}"); return 0.0

    def generate_verification_report(self, copyright_info, similarity, expected_owner, status_override=None):
        report = {'timestamp': datetime.datetime.now().isoformat(),'similarity_score': similarity,'expected_owner_match': False}
        auth_threshold = 0.75 # Adjusted threshold
        if status_override: report['authentication_status'], report['confidence_level'] = status_override, "N/A"
        elif similarity >= auth_threshold: report['authentication_status'], report['confidence_level'] = 'AUTHENTIC', self.get_confidence_level(similarity)
        else: report['authentication_status'], report['confidence_level'] = 'SUSPICIOUS', self.get_confidence_level(similarity)
        if copyright_info:
            report.update({'copyright_id': copyright_info['copyright_id'],'registered_owner': copyright_info['owner_name'],'registration_date': copyright_info['registration_date']})
            if expected_owner and expected_owner.lower() in copyright_info['owner_name'].lower(): report['expected_owner_match'] = True
        elif report['authentication_status'] not in ["ERROR", "DB_EMPTY"]: report['message'] = "No matching copyright found in database."
        return report

    def get_confidence_level(self, similarity):
        if similarity >= 0.9: return "VERY HIGH"
        elif similarity >= 0.75: return "HIGH"    # Adjusted
        elif similarity >= 0.60: return "MEDIUM"  # Adjusted
        elif similarity >= 0.45: return "LOW"     # Adjusted
        else: return "VERY LOW"

    def log_verification(self, image_identifier, report): # Changed path to identifier
        self.verification_log.append({ 'image_identifier': image_identifier,'timestamp': report['timestamp'],'status': report['authentication_status'],'similarity': report['similarity_score'],'confidence': report['confidence_level'],'copyright_id': report.get('copyright_id', 'N/A'),'owner_match': report.get('expected_owner_match', 'N/A')})

    def display_verification_report(self, report):
        print("\n" + "="*70 + "\n🔍 AUTHENTICITY VERIFICATION REPORT\n" + "="*70)
        status_icon = "✅" if report['authentication_status'] == 'AUTHENTIC' else "❌"
        print(f"🕒 Time: {report['timestamp']}")
        print(f"📊 Similarity (Correlation): {report['similarity_score']:.4f}")
        print(f"🎯 Confidence: {report['confidence_level']}")
        print(f"🔐 Status: {status_icon} {report['authentication_status']}")
        if 'copyright_id' in report:
            print(f"📄 Copyright ID Found: {report['copyright_id']}")
            print(f"👤 Registered Owner: {report['registered_owner']}")
            owner_match_status = 'CONFIRMED' if report['expected_owner_match'] else 'NOT MATCHED'
            print(f"👤 Expected Owner Match: {owner_match_status}")
        elif report['authentication_status'] not in ["ERROR", "DB_EMPTY"]: print("❌ No valid copyright match found.")
        elif 'message' in report: print(f"ℹ️ Message: {report['message']}")
        print("="*70)

    def display_database(self):
        print("\n" + "="*50 + "\n📊 COPYRIGHT DATABASE\n" + "="*50)
        if not self.database: print("   Database is empty.")
        else:
            for i, (cid, data) in enumerate(self.database.items()):
                info = data.get('copyright_info', {})
                print(f"   {i+1}. ID: {cid}\n      👤 Owner: {info.get('owner_name', 'N/A')}\n      📅 Registered: {info.get('registration_date', 'N/A')}")
                if i < len(self.database) - 1: print("   " + "-"*40)
        print("="*50)

    def display_verification_log(self):
         print("\n" + "="*70 + "\n📜 AUTHENTICITY VERIFICATION LOG\n" + "="*70)
         if not self.verification_log: print("   Log is empty.")
         else:
             for i, entry in enumerate(self.verification_log):
                 id_display = entry['image_identifier'] # Use identifier
                 if len(id_display) > 50: id_display = "..." + id_display[-47:]
                 print(f"   {i+1}. Timestamp: {entry['timestamp']}\n      🖼️ Image ID: {id_display}\n      🔐 Status: {entry['status']}\n      📊 Similarity: {entry['similarity']:.4f}\n      🎯 Confidence: {entry['confidence']}\n      📄 Found ID: {entry['copyright_id']}\n      👤 Owner Match: {entry['owner_match']}")
                 if i < len(self.verification_log) - 1: print("   " + "-"*60)
         print("="*70)

# --- End of Corrected Class (v3) ---

In [4]:
class WatermarkingGUI:
    """Interactive GUI for the watermarking system"""

    def __init__(self, system_instance): # Accept an instance of the system
        self.system = system_instance # Use the passed instance
        self.setup_ui()

    def setup_ui(self):
        """Setup the user interface"""

        # --- Widgets ---
        self.upload_host = widgets.FileUpload(description='Host Image:', accept='image/*', multiple=False, layout=widgets.Layout(width='auto'))
        self.owner_name = widgets.Text(description='Owner Name:', placeholder='Enter copyright owner name', layout=widgets.Layout(width='auto'))
        self.contact_info = widgets.Text(description='Contact Info:', placeholder='Email or phone', layout=widgets.Layout(width='auto'))
        self.rights_info = widgets.Text(description='Rights:', value='All rights reserved', layout=widgets.Layout(width='auto'))
        self.register_btn = widgets.Button(description='Register Copyright', button_style='success', icon='copyright', layout=widgets.Layout(width='auto'))
        self.img_output_reg = widgets.Output(layout={'border': '1px solid black', 'margin_top': '10px', 'height': '200px'}) # Output for reg images

        self.upload_test = widgets.FileUpload(description='Test Image:', accept='image/*', multiple=False, layout=widgets.Layout(width='auto'))
        self.verify_btn = widgets.Button(description='Verify Authenticity', button_style='info', icon='search', layout=widgets.Layout(width='auto'))

        self.summary_btn = widgets.Button(description='Database Summary', button_style='warning', icon='database', layout=widgets.Layout(width='auto'))
        self.log_btn = widgets.Button(description='View Log', button_style='primary', icon='history', layout=widgets.Layout(width='auto'))

        self.output_status = widgets.Output(layout={'border': '1px solid lightgray', 'padding': '10px', 'margin_top': '10px'}) # Main output area

        # --- Callbacks ---
        self.register_btn.on_click(self.on_register_click)
        self.verify_btn.on_click(self.on_verify_click)
        self.summary_btn.on_click(self.on_summary_click)
        self.log_btn.on_click(self.on_log_click)

        # --- Layout ---
        reg_box = widgets.VBox([
            widgets.HTML("<h3>📝 Copyright Registration</h3>"),
            widgets.HBox([self.upload_host, self.owner_name]),
            widgets.HBox([self.contact_info, self.rights_info]),
            self.register_btn,
            self.img_output_reg # Add image output here
        ], layout=widgets.Layout(width='auto'))

        ver_box = widgets.VBox([
            widgets.HTML("<h3>🔍 Authenticity Verification</h3>"),
            self.upload_test,
            self.verify_btn
        ], layout=widgets.Layout(width='auto'))

        info_box = widgets.VBox([
            widgets.HTML("<h3>📊 System Information</h3>"),
            self.summary_btn,
            self.log_btn
        ], layout=widgets.Layout(width='auto'))

        # Use AppLayout for better structure
        app_layout = widgets.AppLayout(
            header=widgets.HTML("<h2>Enhanced Watermarking System Interface</h2>"),
            left_sidebar=reg_box,
            center=ver_box,
            right_sidebar=info_box,
            footer=self.output_status,
            pane_widths=['35%', '30%', '35%'], # Adjust widths as needed
            grid_gap='10px'
        )

        display(app_layout)

    # --- Event Handlers ---
    def get_uploaded_content(self, upload_widget):
        """ Safely get content from FileUpload """
        if not upload_widget.value:
            return None, None # Return None for content and name
        # The value is a dict where keys are filenames and values are dicts with metadata
        uploaded_file_info = list(upload_widget.value.values())[0]
        file_name = uploaded_file_info['metadata']['name']
        content = uploaded_file_info['content']
        return BytesIO(content), file_name # Return BytesIO and filename


    def on_register_click(self, btn):
        """Handle copyright registration"""
        with self.output_status: # Use status output area
            clear_output(wait=True)
            print("Processing registration...")

        host_content, host_filename = self.get_uploaded_content(self.upload_host)
        owner = self.owner_name.value
        contact = self.contact_info.value
        rights = self.rights_info.value

        with self.output_status:
            if not host_content:
                print("❌ Please upload a host image first.")
                return
            if not owner:
                print("❌ Please enter owner name.")
                return

            try:
                # Pass BytesIO directly to register_copyright
                watermarked_img, copyright_info = self.system.register_copyright(
                    host_content, owner, contact, rights
                )

                if watermarked_img is None: # Handle registration failure
                    print("❌ Registration process failed (check logs above).")
                    return

                # Display results in the specific image output area for registration
                with self.img_output_reg:
                    clear_output(wait=True)
                    plt.figure(figsize=(10, 4)) # Adjusted size for two images
                    # Original image (re-process for display)
                    host_content.seek(0) # Reset stream position
                    original_display = self.system.preprocess_image(host_content)
                    plt.subplot(1, 2, 1)
                    plt.imshow(original_display, cmap='gray')
                    plt.title(f'Original: {host_filename}')
                    plt.axis('off')

                    # Watermarked image
                    plt.subplot(1, 2, 2)
                    plt.imshow(watermarked_img, cmap='gray')
                    plt.title('Watermarked Image')
                    plt.axis('off')

                    plt.tight_layout()
                    plt.show()

                # Confirmation message in the main status area
                print(f"✅ Copyright registered successfully for '{host_filename}'!")
                print(f"📄 Copyright ID: {copyright_info['copyright_id']}")
                # Optionally offer download link (more complex)

            except Exception as e:
                print(f"❌ Registration failed: {str(e)}")
                import traceback
                traceback.print_exc()

            # Clear the upload widget value after processing
            self.upload_host.value = ()


    def on_verify_click(self, btn):
        """Handle authenticity verification"""
        with self.output_status:
            clear_output(wait=True)
            print("Processing verification...")

        test_content, test_filename = self.get_uploaded_content(self.upload_test)

        with self.output_status:
            if not test_content:
                print("❌ Please upload an image to verify.")
                return

            try:
                # Pass BytesIO directly to verify_authenticity
                report, copyright_info, _ = self.system.verify_authenticity(
                    test_content, self.owner_name.value if self.owner_name.value else None # Use owner name if provided for match check
                )

                # Display the text report
                print(f"\n--- Verification Result for '{test_filename}' ---")
                self.system.display_verification_report(report)

                # No spatial watermark to display, just show the test image if needed
                with self.img_output_reg: # Reuse registration image output or create a new one
                     clear_output(wait=True)
                     plt.figure(figsize=(5, 5))
                     test_content.seek(0) # Reset stream position
                     test_img_display = self.system.preprocess_image(test_content)
                     plt.imshow(test_img_display, cmap='gray')
                     plt.title(f'Test Image: {test_filename}')
                     plt.axis('off')
                     plt.show()


            except Exception as e:
                print(f"❌ Verification failed: {str(e)}")
                import traceback
                traceback.print_exc()

            # Clear the upload widget value after processing
            self.upload_test.value = ()


    def on_summary_click(self, btn):
        """Display database summary"""
        with self.output_status:
            clear_output(wait=True)
            self.system.display_database() # Correct method name

    def on_log_click(self, btn):
        """Display verification log"""
        with self.output_status:
            clear_output(wait=True)
            self.system.display_verification_log() # Correct method name

In [5]:
if 'tested_system' in locals() and tested_system is not None:
    print("🚀 Using existing watermarking system instance...")
    watermarking_system_instance = tested_system
else:
    print("🚀 Initializing new watermarking system instance...")
    watermarking_system_instance = FixedWatermarkingSystem(base_alpha=0.15) # Example alpha

# Create and display the GUI, passing the system instance
print("🎨 Creating GUI...")
gui = WatermarkingGUI(watermarking_system_instance)

🚀 Initializing new watermarking system instance...
🎨 Creating GUI...


AppLayout(children=(HTML(value='<h2>Enhanced Watermarking System Interface</h2>', layout=Layout(grid_area='hea…